# **<mark>Isolation Levels</mark>**

## **<mark>Các tác động có thể gây ra khi truy cập đồng thời</mark>**

Người dùng thao tác dữ liệu có thể gây ảnh hưởng tới những người khác đang thao tác đồng thời trên dữ liệu này. Nếu hệ thống không kiểm soát tốt, người dùng có thể gặp 1 số tác động sau:

- **Lost update:** Trường hợp này xảy ra khi 2 hoặc nhiều transacetion cùng update 1 row từ giá trị ban đầu của nó, Update cuối cùng sẽ ghi đè các update bởi các transaction khác dẫn đến mất dữ liệu. 
- **Uncommitted dependency (dirty read):** Tác động này xảy ra khi 1 transaction A đọc 1 row khi nó đang được update bởi 1 transaction B khác và chưa được commit. Transaction A sẽ đọc dữ liệu vẫn chưa được commit. Ví dụ 1 người A trong tài khoản có 3 triệu và đang thực hiện 1 giao dịch để nạp vào 2 triệu cho tài khoản. Transaction đã chạy update xong tài khoản lên 5 triệu nhưng vẫn còn 1 số thao tác chưa chạy tới và dữ liệu này vẫn chưa được commit. 1 người B khác dùng chung tài khoản thực hiện kiểm tra số dư, và kết quả trả về là 5 triệu. Giao dịch của người A xảy ra sự cố và báo lỗi, dữ liệu được roll back về 3 triệu. Như vậy người B đang nhận được dữ liệu sai. 
- **Inconsistent analysis (nonrepeatable read):** Trường hợp này xảy ra khi 1 transaction A đọc 1 đơn vị dữ liệu nhiều lần và kết quả khác nhau giữa các lần do giữa thời gian đọc của các lân đó, dữ liệu bị 1 transaction khác commit thay đổi.
- **Phantom reads:** xảy ra khi 2 queries giống hết nhau được thực hiện nhưng list rows kết quả trả về lại khác nhau. Ví dụ, có 2 transaction được thực thi cùng lúc. Hai câu lệnh SELECT trong transaction đầu tiên có thể trả về các kết quả khác nhau vì câu lệnh INSERT trong transaction thứ hai thay đổi dữ liệu được sử dụng bởi cả hai.

## **<mark>Isolation Levels / Mức độ độc lập</mark>**

Isolation levels là các mức cô lập dữ liệu. Mỗi transaction được chỉ định 1 isolation level để chỉ định mức độ mà nó phải được cách ly khỏi các sự sửa đổi dữ liệu được thực hiện bởi các transaction khác. 

Mức độ cô lập dữ liệu thấp giúp tăng khả năng xử lý đồng thời từ đó hiệu suất cũng tăng theo nhưng nó sẽ làm tăng nguy cơ xảy ra các tác động xấu đã nêu ở phần trước. Tùy từng hoàn cảnh, môi trường và yêu cầu của ứng dụng, ta chọn lựa các mức cô lập dữ liệu phù hợp nhất. 

SQL cung cấp các mức isolation levels sau xếp theo thứ tự tăng dần của mức độ cô lập của dữ liệu: **Read Uncommitted, Read Commited, Repeatable Read, Serializable, Snapshot**

**Read uncommitted**

- Đây là mức cô lập thấp nhất. Khi transaction thực hiện ở mức này, các truy vấn vẫn có thể truy nhập vào các bản ghi đang được cập nhật bởi một transaction khác và nhận được dữ liệu tại thời điểm đó mặc dù dữ liệu đó chưa được commit (uncommited data). Điều này sẽ dẫn đến có thể xảy ra _Dirty read_

**Read commited**

- Đây là mức isolation <mark>mặc định</mark>, nếu bạn không đặt gì cả thì transaction sẽ hoạt động ở mức này. Transaction sẽ không đọc được dữ liệu đang được cập nhật mà phải đợi đến khi việc cập nhật thực hiện xong. Vì thế nó tránh được _dirty read_ như ở mức trên nhưng có thể xảy ra _phantom read_

**Repeatable read**

- Mức isolation này hoạt động như mức _read commited_ nhưng nâng thêm một nấc nữa bằng cách ngăn không cho transaction ghi vào dữ liệu đang được đọc bởi một transaction khác cho đến khi transaction khác đó hoàn tất
- Mức isolation này đảm bảo các lệnh đọc trong cùng một transaction cho cùng kết quả, nói cách khác dữ liệu đang được đọc sẽ được bảo vệ khỏi cập nhật bởi các transaction khác. Tuy nhiên nó không bảo vệ được dữ liệu khỏi insert hoặc delete: nếu bạn thay lệnh update ở cửa sổ thứ hai bằng lệnh insert, hai lệnh select ở cửa sổ đầu sẽ cho kết quả khác nhau. Vì thế nó vẫn không tránh được hiện tượng _phantom read_

**Serializable**

- Đây là mức cô lập cao nhất, các transactions hoàn toàn tách biệt với nhau, SQL đặt read + write lock trên dữ liệu cho tới khi transaction kết thúc. Vì thế hiện tượng _phantom read_ sẽ không còn ở mức này.

**Snapshot**

- Mức độ này chỉ có thể sử dụng khi row versioning được bật.
- Mức độ này cũng đảm bảo độ cô lập tương đương với Serializable, nhưng nó hơi khác ở phương thức hoạt động. Khi transaction đang select các bản ghi, nó không khóa các bản ghi này lại mà tạo một bản sao (snapshot) và select trên đó. Vì vậy các transaction khác insert/update lên các bản ghi đó không gây ảnh hưởng đến transaction ban đầu. Tác dụng của nó là giảm blocking giữa các transaction mà vẫn đảm bảo tính toàn vẹn dữ liệu. Tuy nhiên cái giá kèm theo là cần thêm bộ nhớ để lưu bản sao của các bản ghi, và phần bộ nhớ này là cần cho mỗi transaction do đó có thể tăng lên rất lớn

### **Phạm vi các tác động do truy cập đồng thời đối với mỗi isolation level**

![IsolationLevel](Isolation_Levels.png)

## **Demo**

In [ ]:
USE master
GO

CREATE DATABASE TestDB
GO

USE TestDB
GO

CREATE TABLE IsolationTests
(
    Id INT IDENTITY,
    Col1 INT,
    Col2 INT,
    Col3 INT
)

INSERT INTO IsolationTests(Col1,Col2,Col3)
SELECT 1,2,3
UNION ALL SELECT 1,2,3
UNION ALL SELECT 1,2,3
UNION ALL SELECT 1,2,3
UNION ALL SELECT 1,2,3
UNION ALL SELECT 1,2,3
UNION ALL SELECT 1,2,3

Nếu bạn muốn kiểm tra mức độ cách ly mà bạn đang chạy, bạn có thể chạy "DBCC useroptions". Hãy nhớ rằng các mức cách ly là Kết nối / Giao dịch cụ thể nên các truy vấn khác nhau trên cùng một cơ sở dữ liệu thường được chạy dưới các mức cách ly khác nhau.

In [ ]:
DBCC useroptions

**Read Uncommitted**

In [ ]:
-- Query 1
BEGIN TRAN
UPDATE IsolationTests SET Col1 = 2
--Simulate having some intensive processing here with a wait
WAITFOR DELAY '00:00:10'
ROLLBACK

In [ ]:
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SELECT * FROM IsolationTests

--SELECT * FROM IsolationTests WITH(NOLOCK)

**Read Committed**

In [ ]:
-- Query 1
BEGIN TRAN
UPDATE IsolationTests SET Col1 = 2
--Simulate having some intensive processing here with a wait
WAITFOR DELAY '00:00:10'
ROLLBACK

In [ ]:
-- Query 2
SELECT * FROM IsolationTests

In [ ]:
-- Non-repeatable read
--Query 1
USE TestDB
GO

--Transaction isolation level is default - READ COMMITTED
BEGIN TRANSACTION Transaction1

 SELECT *
 FROM [dbo].[IsolationTests]
 WHERE [Col1] = 1
 GO

 WAITFOR DELAY '00:00:10' 

 SELECT *
 FROM [dbo].[IsolationTests]
 WHERE [Col1] = 1

COMMIT

In [ ]:
--Query 2

USE TestDB
GO

UPDATE [dbo].[IsolationTests]
SET [Col1] = 1
WHERE [Id] = 8

**Repeatable Read**

In [ ]:
-- Query 1
SET TRANSACTION ISOLATION LEVEL REPEATABLE READ
BEGIN TRAN
SELECT * FROM IsolationTests
WAITFOR DELAY '00:00:10'
SELECT * FROM IsolationTests
ROLLBACK

In [ ]:
-- Query 2
UPDATE IsolationTests SET Col1 = -1

**Serializable**

In [ ]:
-- Query 1
SET TRANSACTION ISOLATION LEVEL SERIALIZABLE
BEGIN TRAN
SELECT * FROM IsolationTests
WAITFOR DELAY '00:00:10'
SELECT * FROM IsolationTests
ROLLBACK

In [ ]:
-- Query 2
INSERT INTO IsolationTests(Col1,Col2,Col3)
VALUES (100,100,100)